In [10]:
import nltk
import spacy
from spacy import displacy
from nltk import Tree
from nltk.tree import ParentedTree
import es_core_news_sm
#import stanza
#from spacy_stanza import StanzaLanguage

#nlp = es_core_news_sm.load()

nlp = spacy.load('es_core_news_sm')

#snlp = stanza.Pipeline(lang="es")
#nlp = StanzaLanguage(snlp)

In [11]:
def to_nltk_tree(node):
    if node.n_lefts + node.n_rights > 0:
        return Tree(node.orth_ + '-' + node.dep_, [to_nltk_tree(child) for child in node.children])
    else:
        return node.orth_ + '-' + node.dep_

    
def to_nltk_tree1(node):
    if node.n_lefts + node.n_rights > 0:
        return Tree(node.tag_, [to_nltk_tree1(child) for child in node.children])
    else:
        return node.tag_

    
def crear_arbol(texto, nlp):
    document = nlp(texto)
    jc = list(document.sents)[0]
    arbol = to_nltk_tree(jc.root)
    # arbol = ParentedTree.convert(arbol)
    arbol.pretty_print()
    return arbol


def rec1(A):
    if type(A) is nltk.Tree:
        cadena = A.label() 
        for i, B in enumerate(A):
            cadena += '(' + rec1(B) + ')'
        return cadena
    else:
        return A
    
    
def aplicacion(A, diccionario):
    if type(A) is nltk.Tree:
        funcion = diccionario[A.label()]
        argumentos = [aplicacion(x, diccionario) for x in A]
        return funcion(*argumentos)
    else:
        return diccionario[A]

    
def limpiar_texto(texto, stopwords):
#     token = nltk.RegexpTokenizer(r"\w+")
#     tokens = token.tokenize(texto)
#     lemmatizer = nltk.stem.WordNetLemmatizer()
#    snlp = stanza.Pipeline(lang="es")
#    nlp = StanzaLanguage(snlp)
#    texto = nlp(texto)
    lista_texto = texto.split()
#    lista = [token.lemma_ for token in texto]
    lista = lista_texto
    lista = [x for x in lista if x not in stopwords]
    return ' '.join(lista)

# Función que retorna la forma lógica de un átomo
def atomo(texto, nlp, diccionario):
    arbol = crear_arbol(texto, nlp)
    return aplicacion(arbol, diccionario)

# Función para separar por y/o
def prop(texto, nlp, diccionario):
    documento = nlp(texto)
    palabras = [t.orth_ for t in documento]
    for i,token in enumerate(documento):
#        print(i, token.tag_, token)
        if token.tag_ == 'CCONJ':
            form1 = atomo(' '.join(palabras[:i]), nlp, diccionario)
            form2 = atomo(' '.join(palabras[i+1:]), nlp, diccionario)
            if str(token) == 'y':
                conectivo = ' Y '
            elif str(token) == 'o':
                conectivo = ' O '
            else:
                raise Exception('Conectivo desconocido', token)
            return form1 + conectivo + form2
    return atomo(texto, nlp, diccionario)

# Función que incluye condicionales
def traducir(texto, nlp, diccionario):
    documento = nlp(texto)
    palabras = [t.orth_ for t in documento]
    implicacion = False
    coma = False
    entonces = True
    for i,token in enumerate(documento):
        if token.tag_ == "SCONJ":
            ind_si = i
            implicacion = True
        if implicacion:
            if token.tag_ == "PUNCT" and token.orth_ == ",":
                ind_coma = i
                coma = True
            if token.tag_ == "ADV" and token.orth_ == "entonces":
                ind_ent = i
                entonces = True
    if implicacion and (ind_coma == ind_ent - 1):
        Antecedente = prop(' '.join(palabras[ind_si + 1 : ind_coma]), nlp, diccionario)
        Consecuente = prop(' '.join(palabras[ind_ent +1:]), nlp, diccionario)
        return Antecedente + ' -> ' + Consecuente
    return prop(texto, nlp, diccionario)

##Tipos de Entidades
1: Objetos o personas
2: Lugares
3: Tiempos

In [21]:
diccionario = {'camina-ROOT': lambda x: 'CAMINAR(' + str(x) + ')',
               'está-ROOT': lambda x, y: 'ESTAR(' + str(x) + ',' + str(y) + ')', 
               'detesta-ROOT': lambda x, y: 'DETESTAR(' + str(x) + ',' + str(y) + ')', 
               'viaja-ROOT' : lambda x,y,z,w: 'VIAJAR(' + str(x) + ',' + str(y) + ',' + str(z) + ',' + str(w) + ')',
               'tiene-ROOT': lambda x, y: 'TENER(' + str(x) + ',' + str(y) + ')',
               'tener-ROOT': lambda x, y: 'TENER(' + str(x) + ',' + str(y) + ')',
               'comer-ccomp': lambda x: 'COMER(' + str(x) + ')',
               'juan-nsubj':'j1',
               'laptop-nsubj': 'l1',
               'maria-obj':'m1',
               'pizza-obj' : 'p1',
               'laptop-obj': 'l1',
              'Bogotá-nsubj': 'b2',
              'Bogotá-obj': 'b2',
              'Medellín-obj': 'm2',
              'bogota-obj': 'b2',
              'medellin-obj': 'm2',
              'martes-obj': 'm3',
              'miercoles-obj': 'c3',
              }

stopwords = ['la', 'en', 'a', 'una', 'el', 'de']

---

### Primer ejemplo

In [13]:
texto = 'juan tiene una laptop'
texto = limpiar_texto(texto, stopwords)
traducir(texto, nlp, diccionario)

           tiene-ROOT           
     __________|__________       
juan-nsubj            laptop-obj



'TENER(j1,l1)'

---

### Segundo ejemplo

In [14]:
texto = 'juan tiene una laptop y juan está en Bogotá'
texto = limpiar_texto(texto, stopwords)
traducir(texto, nlp, diccionario)

           tiene-ROOT           
     __________|__________       
juan-nsubj            laptop-obj

           está-ROOT             
     __________|__________        
juan-nsubj           Bogotá-nsubj



'TENER(j1,l1) Y ESTAR(j1,b2)'

---

### Tercer ejemplo

In [15]:
texto = 'juan viaja bogota medellin martes'
texto = limpiar_texto(texto, stopwords)
traducir(texto, nlp, diccionario)

           viaja-ROOT                        
     __________|_______________________       
juan-nsubj bogota-obj medellin-obj martes-obj



'VIAJAR(j1,b2,m2,m3)'

---

### Cuarto ejemplo

In [19]:
texto = 'si juan está en Bogotá, entonces la laptop está en Bogotá'
texto = limpiar_texto(texto, stopwords)
traducir(texto, nlp, diccionario)

           está-ROOT             
     __________|__________        
juan-nsubj           Bogotá-nsubj

             está-ROOT             
      ___________|__________        
laptop-nsubj           Bogotá-nsubj



'ESTAR(j1,b2) -> ESTAR(l1,b2)'

---

### Quinto ejemplo

In [17]:
txt = "Si juan tiene una laptop y juan está en Bogotá, entonces la laptop está en Bogotá"
txt = limpiar_texto(txt, stopwords)
traducir(txt, nlp, diccionario)

           tiene-ROOT           
     __________|__________       
juan-nsubj            laptop-obj

           está-ROOT             
     __________|__________        
juan-nsubj           Bogotá-nsubj

             está-ROOT             
      ___________|__________        
laptop-nsubj           Bogotá-nsubj



'TENER(j1,l1) Y ESTAR(j1,b2) -> ESTAR(l1,b2)'

---

In [ ]:
texto = 'juan tiene una laptop. juan viaja de bogota a medellin el martes'
documento = texto.split('. ')
formulas_texto = []
for s in documento:
    txt = limpiar_texto(s, stopwords)
    formulas_texto.append(traducir(str(txt), nlp, diccionario))
print(formulas_texto)

In [90]:
def pregunta_donde(txt):
    pass

# txt = 'dónde VERBO SUJETO?'
txt = 'dónde está el laptop?'


tipo_funcion = type(lambda x: x2)
print(tipo_funcion)
Entidades = [k for k in diccionario.keys() if type(diccionario[k]) != tipo_funcion]
print("Entidades", Entidades)
Lugares = [k for k in Entidades if '2' in diccionario[k]]
print("Lugares", Lugares)


document = nlp(txt)
sujetos = [t.orth_ for t in document if t.tag_ == 'NOUN']
verbs = [t.orth_ for t in document if t.tag_ == 'VERB']
for x2 in Lugares:
    props.append(f'{sujetos[0]} {verbs[0]} {x2}')

print('')
print(props)
# 'dónde está el laptop?' => 'el laptop está x2'

<class 'function'>
Entidades ['juan-nsubj', 'laptop-nsubj', 'maria-obj', 'pizza-obj', 'laptop-obj', 'Bogotá-nsubj', 'Bogotá-obj', 'Medellín-obj', 'bogota-obj', 'x2-obj', 'medellin-obj', 'martes-obj', 'miercoles-obj']
Lugares ['Bogotá-nsubj', 'Bogotá-obj', 'Medellín-obj', 'bogota-obj', 'x2-obj', 'medellin-obj']

['SUJETO VERBO Bogotá-nsubj', 'SUJETO VERBO Bogotá-obj', 'SUJETO VERBO Medellín-obj', 'SUJETO VERBO bogota-obj', 'SUJETO VERBO x2-obj', 'SUJETO VERBO medellin-obj', 'laptop está Bogotá-nsubj', 'laptop está Bogotá-obj', 'laptop está Medellín-obj', 'laptop está bogota-obj', 'laptop está x2-obj', 'laptop está medellin-obj', 'laptop está Bogotá-nsubj', 'laptop está Bogotá-obj', 'laptop está Medellín-obj', 'laptop está bogota-obj', 'laptop está x2-obj', 'laptop está medellin-obj']


In [84]:
texto = 'donde está el libro'
texto = limpiar_texto(texto, stopwords)
document = nlp(texto)
palabras = [(t.orth_, t.tag_, t.dep_) for t in document]
print(palabras)
jc = list(document.sents)[0]
print(jc.root)
arbol = to_nltk_tree(jc.root)
# arbol = ParentedTree.convert(arbol)
arbol.pretty_print()

[('donde', 'PRON', 'obl'), ('está', 'VERB', 'ROOT'), ('libro', 'NOUN', 'nsubj')]
está
          está-ROOT            
     _________|__________       
donde-obl           libro-nsubj



In [27]:
texto = 'la laptop está en x'
texto = limpiar_texto(texto, stopwords)
traducir(texto, nlp, diccionario)

AttributeError: 'str' object has no attribute 'pretty_print'

In [ ]:
preguntas = 'dónde está el laptop el miércoles? dónde está juan el martes?'
documento = preguntas.split('?')
formulas_preguntas = []
for s in documento:
    txt = limpiar_texto(s, stopwords)
    formulas_preguntas.append(traducir(str(txt), nlp, diccionario))
print(formulas_preguntas)

In [ ]:
sentido_comun = 'Si juan tiene una laptop y juan está en Bogotá, entonces la laptop está en Bogotá'
documento = sentido_comun.split('. ')
formulas_sentido_comun = []
for s in documento:
    txt = limpiar_texto(s, stopwords)
    formulas_sentido_comun.append(traducir(str(txt), nlp, diccionario))
print(formulas_sentido_comun)

## Preguntas
Al encontrar preguntas con "Dónde" se debe realizar una lista con las posibles respuestas de lugares que hagan verdadera la proposición